# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686590


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:34,  1.23s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:50,  1.87s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:33,  1.32s/it]

Rendering models:  27%|██▋       | 8/30 [00:06<00:20,  1.06it/s]

...

Rendering models:  67%|██████▋   | 20/30 [00:06<00:02,  3.73it/s]

Rendering models:  73%|███████▎  | 22/30 [00:06<00:01,  4.84it/s]

Rendering models:  83%|████████▎ | 25/30 [00:06<00:00,  6.45it/s]

Rendering models:  97%|█████████▋| 29/30 [00:07<00:00,  8.25it/s]

richiemac                             0.000143
CThomas                               0.000192
Dynamatt                              0.000208
kjkavene                              0.000193
rnevils                               0.000144
jmfranci                              0.000311
tosnyder                              0.000132
not-logged-in-ef704c739eb8412f355e    0.000280
msalin03                              0.000115
not-logged-in-61a0765eeab9a5074a2a    0.000112
not-logged-in-7f597f4ddc81398b3439    0.000425
Camocub7                              0.000216
not-logged-in-764a91c7a7d674792cd1    0.364277
mclainjamiel                          0.000156
withadnotat                           0.000159
pangeli5                              0.000142
Planetme                              0.003256
41Letters                             0.000080
Lavadude                              0.022295
not-logged-in-04ed9ae880f18762cd9e    0.006353
clars915                              0.000150
equidad1     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())